# ENSO diversity over time

## Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
import xarray as xr
import pathlib
import cmocean
import os
import cartopy.crs as ccrs

# Import custom modules
import src.utils

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## Funcs

## Load data

In [ ]:
## Load data
_, anom = src.utils.load_consolidated()

## subset for SST
data = anom[["sst", "sst_comp"]]

## subset for early
t_early = dict(time=slice("1850", "1881"))
data_early = data.sel(t_early).compute()

## subset for DJF
data_djf = src.utils.sel_month(
    data_early["sst"].resample({"time": "QS-DEC"}).mean(),
    months=12,
)
data_djf = data_djf.isel(time=slice(1, -1))

## trim to [-5,5]
data_djf_trim = src.utils.reconstruct_fn(
    scores=data_djf,
    components=data["sst_comp"],
    fn=lambda x: x.sel(latitude=slice(-5, 5)),
)

## project back on EOFs
data_djf_trim = (data_djf_trim * data["sst_comp"]).sum(["longitude", "latitude"])

## Compute EOFs

In [ ]:
X = data_djf_trim.stack(sample=["member", "time"])
U, s, Vt = np.linalg.svd(X.transpose("mode", "sample").values, full_matrices=False)

## put results in xr
eofs = xr.Dataset(
    data_vars=dict(
        U=(("mode", "eof_mode"), U),
        V=(("sample", "eof_mode"), Vt.T),
        s=("eof_mode", s),
    ),
    coords=dict(
        mode=X.mode,
        sample=X.sample,
        eof_mode=np.arange(300),
    ),
)

## get spatial patterns
eofs["patterns_trim"] = src.utils.reconstruct_fn(
    scores=eofs["U"],
    components=data["sst_comp"],
    fn=lambda x: x,
)

## get "full" spatial patterns
U_full = (data_djf.stack(sample=["member", "time"]) * eofs["V"]).sum("sample")
# U_full = (data_djf_trim.stack(sample=["member", "time"])  * eofs["V"]).sum("sample")
eofs["patterns"] = src.utils.reconstruct_fn(
    scores=U_full,
    components=data["sst_comp"],
    fn=lambda x: x,
)

## normalize spatial patterns
for p in ["patterns", "patterns_trim"]:
    scale = eofs[p].sel(latitude=slice(-5, 5)).std(["latitude", "longitude"])
    eofs[p] = eofs[p] / scale

In [ ]:
x0 = src.utils.reconstruct_fn(
    scores=data_djf.isel(time=0, member=0),
    components=data["sst_comp"],
    fn=lambda x: x,
)

x1 = src.utils.reconstruct_fn(
    scores=data_djf_trim.isel(time=0, member=0),
    components=data["sst_comp"],
    fn=lambda x: x,
)

In [ ]:
fig = plt.figure(figsize=(6, 10 / 6), layout="constrained")
axs = src.utils.subplots_with_proj(
    fig, nrows=1, ncols=1, format_func=src.utils.plot_setup_pac
)

cp = axs[0, 0].contourf(
    x0.longitude,
    x0.latitude,
    x1 - x0,
    transform=ccrs.PlateCarree(),
    cmap="cmo.balance",
    levels=src.utils.make_cb_range(4, 0.4),
    extend="both",
)

cb = fig.colorbar(cp)

### Plot PC space

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))

ax.scatter(eofs["V"].isel(eof_mode=0), -eofs["V"].isel(eof_mode=1), s=1)
plt.show()

### Plot spatial pattern

In [ ]:
fig = plt.figure(figsize=(6, 10 / 3), layout="constrained")
axs = src.utils.subplots_with_proj(
    fig, nrows=2, ncols=1, format_func=src.utils.plot_setup_pac
)

for j, i in enumerate([0, 1]):
    cp = axs[j, 0].contourf(
        eofs.longitude,
        eofs.latitude,
        eofs["patterns"].isel(eof_mode=i),
        transform=ccrs.PlateCarree(),
        cmap="cmo.balance",
        levels=src.utils.make_cb_range(4, 0.4),
        extend="both",
    )

cb = fig.colorbar(cp)